In [2]:
import sqlite3 as sql
import math
import catalog_utils as cu

#run_phase1('gtw49', '/home/kecai/w49/w49.db')

In [3]:
#UPDATE Phase field for sources that have m5_8 and m8_0 detections and whose
#photometric uncertainities < 0.2 mag in all four IRAC bands
#con is a connection object
#t is the name of the table from which we draw records
def init(con, source):
    cur = con.cursor()
    target = source + '_p1'
    cur.execute('''CREATE TABLE ''' + target + ''' AS 
                   SELECT desig, m3_6, m4_5, m5_8, m8_0, 
                                dm3_6, dm4_5, dm5_8, dm8_0, Type FROM ''' + source + 
                ''' WHERE (m5_8 IS NOT NULL) AND 
                          (m8_0 IS NOT NULL) AND 
                          dm3_6 < ? AND 
                          dm4_5 < ? AND 
                          dm5_8 < ? AND 
                          dm8_0 < ?''', [0.2]*4)
    return target

In [4]:
#apply PAH conditions to extract PAH (Type = 19), update n_Type in t_p1
def extract_19(con, source):
    cur = con.cursor()
    #CREATE TABLE containing PAH galaxies
    target = source + '_19'
    cur.execute('''CREATE TABLE '''+ target + ''' AS 
                   SELECT desig FROM ''' + source + ''' WHERE Type IS NULL AND
                   ((m4_5 - m5_8  < (1.05/1.2) * (m5_8 - m8_0 - 1)) AND
                    (m4_5 - m5_8 < 1.05) AND
                    (m5_8 - m8_0 > 1) AND
                    (m4_5 > 11.5))
                    OR
                   ((m3_6 - m5_8 < (1.5/2) * (m4_5 - m8_0 -1)) AND
                    (m3_6 - m5_8 < 1.5) AND
                    (m4_5 - m8_0 > 1) AND
                    (m4_5 > 11.5))''')
    return target

In [12]:
def extract_29(con, source):
    cur = con.cursor()
    #CREATE TABLE containing AGN
    target = source + '_29'
    cur.execute('''CREATE TABLE '''+ target + ''' AS 
                   SELECT desig FROM ''' + source + ''' WHERE Type IS NULL AND
                  (m4_5 - m8_0 > 0.5 AND m4_5 > 13.5 +(m4_5 - m8_0 - 2.3)/0.4 AND m4_5 > 13.5) 
                  AND
                  (m4_5 > 14 + (m4_5 - m8_0 - 0.5) OR m4_5 > 14.5 - (m4_5 - m8_0 - 1.2)/0.3 OR m4_5 > 14.5)''')
    return target

In [6]:
def extract_9(con, source):
    cur = con.cursor()
    #CREATE TABLE containing shock gas emission 
    target = source + '_9'
    cur.execute('''CREATE TABLE '''+ target + ''' AS 
                   SELECT desig FROM '''+ source + ''' WHERE Type IS NULL AND 
                   (m3_6 - m4_5 > (1.2/0.55)*(m4_5 - m5_8 - 0.3)+0.8) AND
                   (m4_5 - m5_8 <= 0.85) AND
                   (m3_6 - m4_5 > 1.05)''')
    return target

In [7]:
def extract_20(con, source):
    #CREATE TABLE containing pah contaminated apertures
    cur = con.cursor()
    target = source + '_20'
    cur.execute('''CREATE TABLE '''+ target +''' AS
                   SELECT desig FROM '''+ source +''' WHERE Type IS NULL AND
                   m3_6 - m4_5 - SQRT(SQUARE(dm3_6) + SQUARE(dm4_5)) <= 
                     1.4 * (m4_5 - m5_8 + SQRT(SQUARE(dm4_5) + SQUARE(dm5_8)) - 0.7) + 0.15 AND
                   m3_6 - m4_5 - SQRT(SQUARE(dm3_6) + SQUARE(dm4_5)) <= 1.65''')
    return target

In [8]:
def extract_1(con, source):
    cur = con.cursor()
    target = source + '_1'
    cur.execute('''CREATE TABLE '''+ target +''' AS 
                   SELECT desig FROM '''+ source +''' WHERE Type IS NULL AND
                   m4_5 - m5_8 > 0.7 AND
                   m3_6 - m4_5 > 0.7''')
    return target

In [9]:
def extract_2(con, source):
    cur = con.cursor()
    target = source + '_2'
    cur.execute('''CREATE TABLE '''+ target +''' AS 
                   SELECT desig FROM '''+ source +''' WHERE Type IS NULL AND
                   m4_5 - m8_0 - SQRT(SQUARE(dm4_5) + SQUARE(dm8_0)) > 0.5 AND
                   m3_6 - m5_8 - SQRT(SQUARE(dm3_6) + SQUARE(dm5_8)) > 0.35 AND
                   m3_6 - m5_8 + SQRT(SQUARE(dm3_6) + SQUARE(dm5_8)) 
                     <= (0.14/0.04)*(m4_5 - m8_0 - SQRT(SQUARE(dm4_5) + SQUARE(dm8_0))-0.5) + 0.5 AND
                   m3_6 - m4_5 - SQRT(SQUARE(dm3_6) + SQUARE(dm4_5)) > 0.15''')
    return target

In [10]:
#requires two string arguments (put quotes around arguments!)
#master: name of the master table where all data is stored
#constr: connection string to the database
def run_phase1(master, constr):
    con = sql.connect(constr)
    
    #add math udf to sqlite
    cu.add_math(con)
    #initialize and create a working copy for phase1 containing desig, four IRAC channel mags and Type
    wc = init(con, master) 
    cu.update_phase(con, master, wc, 1)
    
    #extract and update PAH in the working copy
    c19 = extract_19(con, wc) 
    cu.update_type(con, wc, c19, 19)
    cu.update_type(con, master, c19, 19)
    
    #extract and update AGN
    c29 = extract_29(con, wc)
    cu.update_type(con, wc, c29, 29)
    cu.update_type(con, master, c29, 29)
    
    c9 = extract_9(con, wc)
    cu.update_type(con, wc, c9, 9)
    cu.update_type(con, master, c9, 9)
    
    c20 = extract_20(con, wc)
    cu.update_type(con, wc, c20, 20)
    cu.update_type(con, master, c20, 20)
    
    c1 = extract_1(con, wc)
    cu.update_type(con, wc, c1, 1)
    cu.update_type(con, master, c1, 1)
    
    c2 = extract_2(con, wc)
    cu.update_type(con, wc, c2, 2)
    cu.update_type(con, master, c2, 2)
    
    return 0

In [14]:
run_phase1('gtw49', '/home/kecai/w49/w49test.db')

0